### This is a notebook that walks through the steps to create the temporal data sheet

### Step 1: Import requirements:

In [1]:
import defi.defi_tools as dft
import pandas as pd
import numpy as np
import io
import requests
import xlsxwriter
from datetime import datetime, date 
import json
#import boto3



### Step 2: Create a dataframe from llama API with the necessary attributes

In [7]:
##llama API
df = dft.getProtocols()
df['address']=df['address'].str.lower()

## Remove unnecessary* factors
#factors_needed=df.iloc[:,[1,2,7,11,12,16,18,27]].reset_index()
factors_needed=df.filter(['name','address','symbol','audits','category','chains','oracles','tvl','forkedFrom'], axis=1).reset_index()
#Create a new SolaceID
factors_needed['SolaceID'] = factors_needed['name'].str.split(' ').str[0]
factors_needed['SolaceID']=factors_needed['SolaceID'].str.lower()


# Array with only ethereum protocols
ethProtocols=[]
for i in factors_needed['chains']:
    if 'Ethereum' in i:
        ethProtocols.append(i)

#First 5 values for demonstration
factors_needed.head()

,name,address,symbol,audits,category,chains,oracles,tvl,forkedFrom,SolaceID
0,Curve,0xd533a949740bb3306d119cc777fa900ba034cd52,CRV,2,Dexes,"[Ethereum, Avalanche, Fantom, Polygon, Arbitru...",[Chainlink],2.003971e+10,NaN,curve
1,MakerDAO,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,MKR,2,CDP,[Ethereum],[Maker],1.796263e+10,NaN,makerdao
2,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,Lending,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.409376e+10,NaN,aave
3,Convex Finance,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,CVX,2,Yield,[Ethereum],NaN,1.384261e+10,NaN,convex
4,WBTC,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,2,Bridge,[Ethereum],[Chainlink],1.190942e+10,NaN,wbtc


### Step 3: Get Zapper Api data from these endpoints:

Zapper addresses:

https://api.zapper.fi/v1/prices?api_key=96e0cc51-a62e-42ca-acee-910ea7d2a241

Zapper protocol attributes:

https://api.zapper.fi/v1/apps?api_key=96e0cc51-a62e-42ca-acee-910ea7d2a241

In [8]:
## Address Endpoint
url_address = r'https://api.zapper.fi/v1/prices?api_key=96e0cc51-a62e-42ca-acee-910ea7d2a241' 
urlData_address = requests.get(url_address).content
zapperAddresses = pd.read_json(io.StringIO(urlData_address.decode('utf-8')))
## Attribute Endpoint
url_attributes = r'https://api.zapper.fi/v1/apps?api_key=96e0cc51-a62e-42ca-acee-910ea7d2a241' 
urlData_attributes = requests.get(url_attributes).content
zapperAttributes = pd.read_json(io.StringIO(urlData_attributes.decode('utf-8')))

f = open('../governance/reference/mappingTablesSolace.json')
lookupTables = json.loads(f.read())
f.close()
#reference=pd.read_json('../governance/reference/mappingTablesSolace.json')
categories=pd.DataFrame(lookupTables['categorySolaceLookup'])

def lookup_categorySolace(category):
    if pd.isna(category):
        return 'unknown'
    elif category not in list(categories['category']):
        return 'other'
    else:
        return category

zapperAttributes['tags']=zapperAttributes['tags'].str.get(0)
zapperAttributes['categorySolace'] = zapperAttributes['tags'].apply(lookup_categorySolace)
zapperAttributes

## Merge attrubutes from both llama and zapper
zapperAttributes['SolaceID'] = zapperAttributes['name'].str.split(' ').str[0]
zapperAttributes['SolaceID']=zapperAttributes['SolaceID'].str.lower()
combinedtable1=pd.merge(factors_needed,zapperAttributes,left_on='SolaceID', right_on='SolaceID', how='left')
## Merge to get more factors from zapper
combinedTable2=pd.merge(combinedtable1,zapperAddresses,left_on='address', right_on='address', how='left')
#pd.set_option('display.max_rows', None, 'display.max_columns', None)




### Step 4: Get launch dates from Dune and merge with Table *WIP 

In [9]:
DuneData=pd.read_csv('../temporalDB/launch_dates.csv',names=["address", "LaunchDate"])
#Skip these statements if you want all protocols
finalTable = pd.merge(combinedTable2,DuneData,left_on='address', right_on='address', how='left')
#Removing more irrelevent columns
finalTable=finalTable.drop(columns=['category','symbol_y', 'hide','canExchange','primaryColor','token','decimals','url','groups','price','supportedNetworks','name_y','disabled'])
finalTable=finalTable.rename(columns={'symbol_x':'symbol','id':'appId','name_x':'name'})
finalTable=finalTable.loc[finalTable['appId'].notnull()]
finalTable=finalTable.loc[finalTable['chains'].isin(ethProtocols)]

finalTable.head()

,name,address,symbol,audits,chains,oracles,tvl,forkedFrom,SolaceID,appId,tags,compatibleAddressFormats,categorySolace,LaunchDate
0,Curve,0xd533a949740bb3306d119cc777fa900ba034cd52,CRV,2,"[Ethereum, Avalanche, Fantom, Polygon, Arbitru...",[Chainlink],2.003971e+10,NaN,curve,curve,liquidity-pool,"{'arbitrum': 'evm', 'avalanche': 'evm', 'ether...",liquidity-pool,2020-08-12 22:17
2,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.409376e+10,NaN,aave,aave-amm,lending,{'ethereum': 'evm'},lending,2020-09-24 18:06
3,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.409376e+10,NaN,aave,aave-safety-module,liquidity-pool,{},liquidity-pool,2020-09-24 18:06
4,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.409376e+10,NaN,aave,aave,lending,{'ethereum': 'evm'},lending,2020-09-24 18:06
5,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.409376e+10,NaN,aave,aave-v2,lending,"{'avalanche': 'evm', 'ethereum': 'evm', 'polyg...",lending,2020-09-24 18:06


### Step 5: Create the current age column 

In [10]:
#Current Date
now = pd.Timestamp('now')
finalTable['LaunchDate'] = pd.to_datetime(finalTable['LaunchDate'], format="%Y-%m-%d %H:%M")    
finalTable['age'] = (now - finalTable['LaunchDate']).astype('<m8[M]')    #Age in months
finalTable

,name,address,symbol,audits,chains,oracles,tvl,forkedFrom,SolaceID,appId,tags,compatibleAddressFormats,categorySolace,LaunchDate,age
0,Curve,0xd533a949740bb3306d119cc777fa900ba034cd52,CRV,2,"[Ethereum, Avalanche, Fantom, Polygon, Arbitru...",[Chainlink],2.003971e+10,NaN,curve,curve,liquidity-pool,"{'arbitrum': 'evm', 'avalanche': 'evm', 'ether...",liquidity-pool,2020-08-12 22:17:00,17.0
2,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.409376e+10,NaN,aave,aave-amm,lending,{'ethereum': 'evm'},lending,2020-09-24 18:06:00,16.0
3,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.409376e+10,NaN,aave,aave-safety-module,liquidity-pool,{},liquidity-pool,2020-09-24 18:06:00,16.0
4,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.409376e+10,NaN,aave,aave,lending,{'ethereum': 'evm'},lending,2020-09-24 18:06:00,16.0
5,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.409376e+10,NaN,aave,aave-v2,lending,"{'avalanche': 'evm', 'ethereum': 'evm', 'polyg...",lending,2020-09-24 18:06:00,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,ARCx,eth:0x1321f1f1aa541a56c31682c57b80ecfccd9bb288,ARCx,0,[Ethereum],NaN,0.000000e+00,NaN,arcx,arcx,yield-aggregator,{'ethereum': 'evm'},yield-aggregator,NaT,NaN
1178,The Sandbox,0x3845badade8e6dff049820680d1f14bd3903a5d0,SAND,0,"[Ethereum, Polygon]",NaN,0.000000e+00,NaN,the,the-graph,liquidity-pool,{'ethereum': 'evm'},liquidity-pool,NaT,NaN
1209,LooksRare,0xf4d2888d29d722226fafa5d9b24f9164c092421e,LOOKS,0,[Ethereum],NaN,0.000000e+00,NaN,looksrare,looksrare,NaN,{'ethereum': 'evm'},unknown,NaT,NaN
1220,The Open DAO SOS,0x3b484b82567a09e2588a13d54d032153f0c0aee0,SOS,2,[Ethereum],NaN,0.000000e+00,NaN,the,the-graph,liquidity-pool,{'ethereum': 'evm'},liquidity-pool,NaT,NaN


### Step 6: Creating excel table and formatting

In [11]:


writer = pd.ExcelWriter('../temporalDB/TemporalDataset.xlsx', engine='xlsxwriter')

finalTable.to_excel(writer, sheet_name='TemporalData', startrow=1, header=False, index=False)

workbook = writer.book
worksheet = writer.sheets['TemporalData']

(max_row, max_col) = finalTable.shape

#Column Headers
column_settings = []
for header in finalTable.columns:
    column_settings.append({'header': header})

# Add the table.
worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

# Make the columns wider for clarity.
worksheet.set_column(0, max_col - 1, 12)
worksheet.set_column('B:B', 40)
worksheet.set_column('E:E', 40)
worksheet.set_column('H:H', 40)
worksheet.set_column('K:K', 30)


writer.save()


### Step 7: Deploy to AWS *WIP

### Step 8: Automatic updates *WIP